In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!kaggle datasets download -d ashkhagan/figshare-brain-tumor-dataset

 99% 829M/839M [00:07<00:00, 100MB/s]
100% 839M/839M [00:07<00:00, 120MB/s]


In [3]:
! unzip figshare-brain-tumor-dataset.zip

Archive:  figshare-brain-tumor-dataset.zip
  inflating: dataset/README.txt      
  inflating: dataset/cvind.mat       
  inflating: dataset/data/1.mat      
  inflating: dataset/data/10.mat     
  inflating: dataset/data/100.mat    
  inflating: dataset/data/1000.mat   
  inflating: dataset/data/1001.mat   
  inflating: dataset/data/1002.mat   
  inflating: dataset/data/1003.mat   
  inflating: dataset/data/1004.mat   
  inflating: dataset/data/1005.mat   
  inflating: dataset/data/1006.mat   
  inflating: dataset/data/1007.mat   
  inflating: dataset/data/1008.mat   
  inflating: dataset/data/1009.mat   
  inflating: dataset/data/101.mat    
  inflating: dataset/data/1010.mat   
  inflating: dataset/data/1011.mat   
  inflating: dataset/data/1012.mat   
  inflating: dataset/data/1013.mat   
  inflating: dataset/data/1014.mat   
  inflating: dataset/data/1015.mat   
  inflating: dataset/data/1016.mat   
  inflating: dataset/data/1017.mat   
  inflating: dataset/data/1018.mat   
  infla

In [2]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Activation
import cv2
from tensorflow import keras

In [3]:
datadir="/content/dataset/data"
N=3064
traindata=[]

for i in range(1,N+1):
    filename=str(i)+".mat"
    data=h5py.File(os.path.join(datadir,filename),"r")
    traindata.append(data)

    if i%100==0:
      print(filename)

100.mat
200.mat
300.mat
400.mat
500.mat
600.mat
700.mat
800.mat
900.mat
1000.mat
1100.mat
1200.mat
1300.mat
1400.mat
1500.mat
1600.mat
1700.mat
1800.mat
1900.mat
2000.mat
2100.mat
2200.mat
2300.mat
2400.mat
2500.mat
2600.mat
2700.mat
2800.mat
2900.mat
3000.mat


In [4]:
import random

random.shuffle(traindata)


*   cjdata.label : 1 for meningioma, 2 for glioma, 3 for pituitary tumor
*   cjdata.image : image data

train test split and adding 1 extra dims for image to be flatten in 2d plane 


In [5]:
train_images = []
train_labels = []
test_images = []
test_labels = []
temp=round(4*3064/5)

for i in range(temp):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    train_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    train_labels.append(label)

for i in range(temp,3064):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    test_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    test_labels.append(label) 
  

In [ ]:
# do not run now just for refrence
"""images = []
labels = []
filename = None
for filename in range(1,3064):
  with h5py.File('/content/dataset/data/{}.mat'.format(filename), 'r') as f:
    img = f['cjdata']['image']
    label = f['cjdata']['label'][0][0]
    labels.append(int(label))
    img = np.array(img, dtype=np.float32)
    images.append(img)
print("{} files successfully saved".format(filename))"""

3063 files successfully saved


In [6]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

To check if all the arrays are consistent

In [7]:
print("Train images shape",train_images.shape)
print("Train labels shape",train_labels.shape)
print("Testimages shape",test_images.shape)
print("Test labels shape",test_labels.shape)

Train images shape (2437, 512, 512, 1)
Train labels shape (2437,)
Testimages shape (612, 512, 512, 1)
Test labels shape (612,)


In [8]:
train_images[500]

array([[[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [2],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [5],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [5],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [4],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [4],
        ...,
        [0],
        [0],
        [0]]], dtype=int16)

In [9]:
ini_input=keras.Input(shape=(512,512,1),name="image")

x1=layers.Conv2D(64,(22,22),strides=2)(ini_input)
x1=layers.MaxPooling2D((4,4))(x1)
x1=layers.BatchNormalization()(x1)

x2=layers.Conv2D(128,(11,11),strides=2,padding="same")(x1)
x2=layers.MaxPooling2D((2,2))(x2)
x2=layers.BatchNormalization()(x2)

x3=layers.Conv2D(256,(7,7),strides=2,padding="same")(x2)
x3=layers.MaxPooling2D((2,2))(x3)
x3=layers.BatchNormalization()(x3)

x4=layers.Flatten()(x3)
x4=layers.Activation("relu")(x4)

x5=layers.Dense(1024,"relu")(x4)

x6=layers.Dense(256,"relu")(x5)

x9=layers.Dense(3)(x6)
pred=layers.Activation("softmax")(x9)


In [10]:
model=keras.Model(inputs=ini_input,outputs=pred)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 512, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 246, 246, 64)      31040     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 61, 61, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 128)       991360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 128)      0     

In [11]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(train_images,train_labels,epochs=10,batch_size=32,shuffle = False)

Epoch 1/10
77/77 [==============================] - 23s 151ms/step - loss: 0.8741 - accuracy: 0.6902
Epoch 2/10
77/77 [==============================] - 11s 147ms/step - loss: 0.5560 - accuracy: 0.7563
Epoch 3/10
77/77 [==============================] - 11s 149ms/step - loss: 0.4598 - accuracy: 0.7936
Epoch 4/10
77/77 [==============================] - 12s 151ms/step - loss: 0.3862 - accuracy: 0.8281
Epoch 5/10
77/77 [==============================] - 12s 151ms/step - loss: 0.3213 - accuracy: 0.8691
Epoch 6/10
77/77 [==============================] - 11s 149ms/step - loss: 0.2580 - accuracy: 0.8974
Epoch 7/10
77/77 [==============================] - 11s 148ms/step - loss: 0.2055 - accuracy: 0.9175
Epoch 8/10
77/77 [==============================] - 11s 147ms/step - loss: 0.1632 - accuracy: 0.9348
Epoch 9/10
77/77 [==============================] - 11s 147ms/step - loss: 0.1566 - accuracy: 0.9462
Epoch 10/10
77/77 [==============================] - 11s 148ms/step - loss: 0.1884 - accura

Model evaluation

In [12]:
result=model.evaluate(test_images,test_labels)
print(result)

20/20 [==============================] - 2s 66ms/step - loss: 0.4075 - accuracy: 0.8562
[0.40752390027046204, 0.8562091588973999]


Testing predictions class probablity

In [33]:
label_pred_keras = model.predict(test_images)
print(label_pred_keras)

20/20 [==============================] - 1s 51ms/step
[[1.3854991e-03 8.7551081e-05 9.9852693e-01]
 [1.7703304e-09 1.0000000e+00 2.4155493e-09]
 [2.8544087e-05 9.9997139e-01 7.3071497e-08]
 ...
 [5.9254390e-01 4.0737358e-01 8.2498802e-05]
 [2.7155843e-06 1.1921464e-08 9.9999726e-01]
 [3.3329934e-04 9.9964643e-01 2.0289164e-05]]


In [35]:
label_pred_keras = model.predict(test_images[80][None,...])
print(label_pred_keras)

1/1 [==============================] - 0s 17ms/step
[[4.3335562e-03 7.0918584e-04 9.9495727e-01]]


Final class output for single image prediction

In [36]:
final_class = np.unravel_index(label_pred_keras.argmax(),label_pred_keras.shape)
if final_class[1] == 0:
  print("meningioma")
elif final_class[1] == 1:
  print("glioma")
else:
  print("pituitary")

pituitary


In [37]:
model.save('AlexNet_v2.h5')

In [38]:
!cp "/content/AlexNet_v2.h5" "/content/drive/MyDrive/Final_year_project/"